In [28]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
import matplotlib.pyplot as plt

import tensorflow as tf
import numpy as np
import pandas as pd
import sqlalchemy

import sklearn.metrics as metrics
from tensorflow.python.data import Dataset

In [29]:
#getting movies dataset using Pandas from movie_metadata provided by Prof Zhang
movies_dataframe =  pd.read_csv("movie_metadata.csv", sep=",")
movies_dataframe

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0
5,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Action|Adventure|Sci-Fi,...,738.0,English,USA,PG-13,263700000.0,2012.0,632.0,6.6,2.35,24000
6,Color,Sam Raimi,392.0,156.0,0.0,4000.0,James Franco,24000.0,336530303.0,Action|Adventure|Romance,...,1902.0,English,USA,PG-13,258000000.0,2007.0,11000.0,6.2,2.35,0
7,Color,Nathan Greno,324.0,100.0,15.0,284.0,Donna Murphy,799.0,200807262.0,Adventure|Animation|Comedy|Family|Fantasy|Musi...,...,387.0,English,USA,PG,260000000.0,2010.0,553.0,7.8,1.85,29000
8,Color,Joss Whedon,635.0,141.0,0.0,19000.0,Robert Downey Jr.,26000.0,458991599.0,Action|Adventure|Sci-Fi,...,1117.0,English,USA,PG-13,250000000.0,2015.0,21000.0,7.5,2.35,118000
9,Color,David Yates,375.0,153.0,282.0,10000.0,Daniel Radcliffe,25000.0,301956980.0,Adventure|Family|Fantasy|Mystery,...,973.0,English,UK,PG,250000000.0,2009.0,11000.0,7.5,2.35,10000


In [30]:
#getting actors dataset using Pandas from BuzzFeed
actors_dataframe = pd.read_csv("actor-metrics.csv", sep=",")

In [31]:
#getting biopics dataset using Pandas from 
biopics_dataframe = pd.read_csv("biopics.csv", sep=",", encoding='latin-1')

In [32]:
#renaming the lead actors column name for each table to have the same name 'lead_actors'
movies_dataframe = movies_dataframe.rename(columns={'actor_1_name': 'lead_actors'})
actors_dataframe = actors_dataframe.rename(columns={'actor': 'lead_actors'})
biopics_dataframe = biopics_dataframe.rename(columns={'lead_actor_actress': 'lead_actors'})

In [33]:
#Clean Up Part 1: Remove unnecessary columns
#I think we should first eliminate unnecessary columns and then merge, this way the table is easier to look at

In [34]:
#Clean Up Part 2: Merge
movies_dataframe_merged = movies_dataframe.merge(actors_dataframe,on='lead_actors').merge(biopics_dataframe,on='lead_actors')
movies_dataframe_merged
# Something doesn't seem to work. On the right part of the big table there are weird things happening: 
# Another director than at the start, other films? Somehow we can not merge the tables this way I think? 
# Or is this part not about movies but just the actors? Then we really need to get rid of most columns!

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,year_release,box_office,director,number_of_subjects,subject,type_of_subject,race_known,subject_race,person_of_color,subject_sex
0,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2008,$105K,Nicolas Winding Refn,1,Charles Bronson,Criminal,Unknown,NaN,0,Male
1,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2008,$105K,Nicolas Winding Refn,1,Charles Bronson,Criminal,Unknown,NaN,0,Male
2,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2008,$105K,Nicolas Winding Refn,1,Charles Bronson,Criminal,Unknown,NaN,0,Male
3,Color,George Miller,739.0,120.0,750.0,943.0,Charlize Theron,27000.0,153629485.0,Action|Adventure|Sci-Fi|Thriller,...,2008,$105K,Nicolas Winding Refn,1,Charles Bronson,Criminal,Unknown,NaN,0,Male
4,Color,George Miller,739.0,120.0,750.0,943.0,Charlize Theron,27000.0,153629485.0,Action|Adventure|Sci-Fi|Thriller,...,2008,$105K,Nicolas Winding Refn,1,Charles Bronson,Criminal,Unknown,NaN,0,Male
5,Color,George Miller,739.0,120.0,750.0,943.0,Charlize Theron,27000.0,153629485.0,Action|Adventure|Sci-Fi|Thriller,...,2008,$105K,Nicolas Winding Refn,1,Charles Bronson,Criminal,Unknown,NaN,0,Male
6,Color,McG,241.0,103.0,368.0,520.0,Abigail Spencer,27000.0,54758461.0,Action|Comedy|Romance,...,2008,$105K,Nicolas Winding Refn,1,Charles Bronson,Criminal,Unknown,NaN,0,Male
7,Color,McG,241.0,103.0,368.0,520.0,Abigail Spencer,27000.0,54758461.0,Action|Comedy|Romance,...,2008,$105K,Nicolas Winding Refn,1,Charles Bronson,Criminal,Unknown,NaN,0,Male
8,Color,McG,241.0,103.0,368.0,520.0,Abigail Spencer,27000.0,54758461.0,Action|Comedy|Romance,...,2008,$105K,Nicolas Winding Refn,1,Charles Bronson,Criminal,Unknown,NaN,0,Male
9,Color,Stuart Baird,172.0,116.0,53.0,906.0,LeVar Burton,27000.0,43119879.0,Action|Adventure|Sci-Fi|Thriller,...,2008,$105K,Nicolas Winding Refn,1,Charles Bronson,Criminal,Unknown,NaN,0,Male


In [35]:
#Clean Up Part 3: Remove duplicates
movies_dataframe_noDuplicates = movies_dataframe_merged.drop_duplicates(subset=['director_name', 'duration', 'lead_actors', 'gross', 'actor_1_facebook_likes'], keep='first', inplace=False)
movies_dataframe_noDuplicates

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,year_release,box_office,director,number_of_subjects,subject,type_of_subject,race_known,subject_race,person_of_color,subject_sex
0,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2008,$105K,Nicolas Winding Refn,1,Charles Bronson,Criminal,Unknown,NaN,0,Male
3,Color,George Miller,739.0,120.0,750.0,943.0,Charlize Theron,27000.0,153629485.0,Action|Adventure|Sci-Fi|Thriller,...,2008,$105K,Nicolas Winding Refn,1,Charles Bronson,Criminal,Unknown,NaN,0,Male
6,Color,McG,241.0,103.0,368.0,520.0,Abigail Spencer,27000.0,54758461.0,Action|Comedy|Romance,...,2008,$105K,Nicolas Winding Refn,1,Charles Bronson,Criminal,Unknown,NaN,0,Male
9,Color,Stuart Baird,172.0,116.0,53.0,906.0,LeVar Burton,27000.0,43119879.0,Action|Adventure|Sci-Fi|Thriller,...,2008,$105K,Nicolas Winding Refn,1,Charles Bronson,Criminal,Unknown,NaN,0,Male
12,Color,Daniel Espinosa,172.0,137.0,79.0,194.0,Fares Fares,27000.0,1206135.0,Crime|Drama|Thriller,...,2008,$105K,Nicolas Winding Refn,1,Charles Bronson,Criminal,Unknown,NaN,0,Male
15,Color,Brian Helgeland,260.0,132.0,241.0,151.0,Paul Anderson,27000.0,1865774.0,Biography|Crime|Drama|History|Thriller,...,2008,$105K,Nicolas Winding Refn,1,Charles Bronson,Criminal,Unknown,NaN,0,Male
18,Color,Gavin O'Connor,317.0,140.0,149.0,581.0,Frank Grillo,27000.0,13651662.0,Drama|Sport,...,2008,$105K,Nicolas Winding Refn,1,Charles Bronson,Criminal,Unknown,NaN,0,Male
21,Color,Guy Ritchie,209.0,114.0,0.0,1000.0,Gerard Butler,27000.0,5694401.0,Action|Crime|Thriller,...,2008,$105K,Nicolas Winding Refn,1,Charles Bronson,Criminal,Unknown,NaN,0,Male
24,Color,NaN,9.0,142.0,NaN,427.0,Jack O'Connell,27000.0,NaN,Drama|Romance,...,2008,$105K,Nicolas Winding Refn,1,Charles Bronson,Criminal,Unknown,NaN,0,Male
27,Color,Matthew Vaughn,149.0,105.0,905.0,466.0,Sally Hawkins,27000.0,2338695.0,Crime|Drama|Thriller,...,2008,$105K,Nicolas Winding Refn,1,Charles Bronson,Criminal,Unknown,NaN,0,Male


In [36]:
#Clean up part 4 : Dataset with necessary columns only
movies_dataframe_noDuplicates = movies_dataframe_noDuplicates[['lead_actors','race_known','person_of_color', 'subject_sex', 'gross','box_office','movie_facebook_likes']]

In [37]:
movies_dataframe_noDuplicates

,lead_actors,race_known,person_of_color,subject_sex,gross,box_office,movie_facebook_likes
0,Tom Hardy,Unknown,0,Male,448130642.0,$105K,164000
3,Tom Hardy,Unknown,0,Male,153629485.0,$105K,191000
6,Tom Hardy,Unknown,0,Male,54758461.0,$105K,18000
9,Tom Hardy,Unknown,0,Male,43119879.0,$105K,0
12,Tom Hardy,Unknown,0,Male,1206135.0,$105K,18000
15,Tom Hardy,Unknown,0,Male,1865774.0,$105K,43000
18,Tom Hardy,Unknown,0,Male,13651662.0,$105K,77000
21,Tom Hardy,Unknown,0,Male,5694401.0,$105K,12000
24,Tom Hardy,Unknown,0,Male,NaN,$105K,0
27,Tom Hardy,Unknown,0,Male,2338695.0,$105K,0
